In [1]:

import pandas as pd

from utils.io import yaml_to_dict,dict_to_yaml

from cleanco import basename

import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict

# from projects.lobbying.postproc import terms_present
import matplotlib.colors as mc

import warnings

warnings.filterwarnings("ignore")
%load_ext jupyter_black

In [11]:
df = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/search_terms/search_term_list.yml"
)

In [ ]:
len(te)

In [12]:
terms = df["search_term_list"]
len(terms)

227

In [13]:
terms = sorted(list(set(terms)))
len(terms)

217

In [16]:
dict_to_yaml(
    {"search_term_list": terms},
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/search_terms/search_term_list.yml",
)

In [16]:
df = pd.read_csv(
    "/Volumes/Samsung_T5/data/lobbying/steel_iron_organizations/ccs_lda_filings_steel_iron_orgs_1.csv"
)

In [5]:
def invert_sector_dict(sectors_path) -> Dict[str, str]:
    """reads in the sector assignment yaml to dict; inverts dict s.t. each company is a key"""
    sector_assignments = yaml_to_dict(sectors_path)

    all_companies = []
    for _, value in sector_assignments.items():
        all_companies = all_companies + value
    # print(all_companies)

    company_sector_dict = {}
    for k, vv in sector_assignments.items():
        for v in vv:
            company_sector_dict = company_sector_dict | {v: k}

    return company_sector_dict

In [6]:
df = pd.read_csv("/Volumes/Samsung_T5/data/lobbying/oct4_compiled_filings.csv")
inverted_sector_dict = invert_sector_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors_updated.yml"
)

In [21]:
term_list = ["hi", "how", "are", "you", "this", "is", "a", "test", "yay"]
search_string_list = []
search_string_chunk_size = 2
for i in range(0, len(term_list), search_string_chunk_size):
    search_string_list.append("OR".join(term_list[i : i + search_string_chunk_size]))

In [22]:
search_string_list

['hiORhow', 'areORyou', 'thisORis', 'aORtest', 'yay']

In [ ]:
for i in df.client_rename.unique():
    if i not in inverted_sector_dict.keys():
        print(f"{i}")

In [3]:
# df = pd.read_csv("/Volumes/Samsung_T5/data/lobbying/oct4_compiled_filings.csv")
df = pd.read_csv(
    "/Volumes/Samsung_T5/data/lobbying/steel_iron_no_ccs/oct9_steel_iron_no_ccs_compiled_filings.csv"
)

In [14]:
df = pd.read_csv(
    "/Volumes/Samsung_T5/data/lobbying/steel_iron_no_ccs/oct9_steel_iron_no_ccs_postprocessed_filings.csv"
)

In [10]:
portland_replace_dict = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/company_name_replacements.yml"
)

df["client_rename"] = [
    portland_replace_dict[x] if x in list(portland_replace_dict.keys()) else r
    for x, r in zip(df.client_name, df.client_rename)
]

inverted_sector_dict = invert_sector_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors_updated.yml"
)
df["sector"] = [inverted_sector_dict[x] for x in df["client_rename"]]

companies = [c.rstrip().lstrip() for c in list(inverted_sector_dict.keys())]
not_assigned = sorted(list(set(df.client_rename.unique()) - set(companies)))
for c in not_assigned:
    print(c)
    # for c in ["FMC"]:
    # names = list(df.loc[df.client_rename == c].client_name.unique())
    # for n in names:
    #    print(f"{n}: {c}")
    # descriptions = list(df.loc[df.client_rename == c].clean_description.unique())
    # for d in descriptions:
    #    print(d)
    print("")

In [ ]:
df.clean_description = df.clean_description.fillna("")
df["ccs"] = [
    1
    if (
        (" ccs" in x.lower())
        | (" ccus" in x.lower())
        | ("capture and utilization" in x.lower())
        | ("carbon capture storage" in x.lower())
        | ("carbon capture and storage" in x.lower())
        | ("co2 capture and sequestration" in x.lower())
        | ("co2 capture sequestration" in x.lower())
        | ("capture transport and storage" in x.lower())
        | ("capture and storage of carbon" in x.lower())
        | ("capture and sequestration of carbon" in x.lower())
        | ("capture sequestration of carbon" in x.lower())
        | ("carbon capture and sequestration" in x.lower())
        | ("carbon capture sequestration" in x.lower())
        | ("co2 capture storage" in x.lower())
        | ("clean hydrogen" in x.lower())
        | ("blue hydrogen" in x.lower())
        | ("beccs" in x.lower())
        | ("hydrogen hub" in x.lower())
        | ("carbon dioxide pipeline" in x.lower())
        | ("co2 pipeline" in x.lower())
        | ("class vi" in x.lower())
    )
    else 0
    for x in df.clean_description
]
df["bio"] = [
    1
    if (
        ("biofuel" in x.lower())
        | ("biomass" in x.lower())
        | ("bioenergy" in x.lower())
        | ("waste to energy" in x.lower())
        | ("wastetoenergy" in x.lower())
        | ("sustainable aviation" in x.lower())
        | ("biodiesel" in x.lower())
    )
    else 0
    for x in df.clean_description
]
df.ccs.value_counts()

In [ ]:
df.loc[df.very_likely_ccs == 1].sector.value_counts()

In [ ]:
for c in sorted(
    list(set((df.loc[(df.sector == "higher education")].client_rename.unique())))
):
    print(f"- {c}")

In [ ]:
for c in sorted(
    list(
        df.loc[
            (df.ccs == 1) & (df.sector == "higher education") & (df.sector != "ccs")
        ].client_rename.unique()
    )
):
    print(f"- {c}")

In [ ]:
df.loc[
    (df.client_rename == "AMERICANS FOR TAX REFORM") & (df.ccs == 1)
].clean_description.values

In [ ]:
for i in list(
    df.loc[df.client_rename == "RENEWABLE ENERGY GROUP"].client_name.unique()
):
    print(f"{i}: ")

In [93]:
import os


def rename_files(directory):
    # Iterate over all files in the specified directory
    for filename in os.listdir(directory):
        # Check if the file is a CSV file
        if filename.endswith(".csv"):
            # Construct the old file path
            old_file_path = os.path.join(directory, filename)

            # Construct the new file name with 'latest_' prefix
            new_file_name = "latest_" + filename

            # Construct the new file path
            new_file_path = os.path.join(directory, new_file_name)

            # Rename the file
            os.rename(old_file_path, new_file_path)
            print(f"Renamed: {filename} to {new_file_name}")

In [ ]:
tmp = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors_updated.yml"
)
he = tmp["chemicals, refining, and plastics"]
he = sorted(list(set(he)))
for h in he:
    print(f"- {h}")

In [ ]:
for c in not_assigned:
    original_names = df.loc[df.client_rename == c].client_name.unique()
    for n in original_names:
        print(f"{n}: {c}")

In [ ]:
# df.to_csv("~/Desktop/postprocessed_ccs.csv")

df = pd.read_csv("/Volumes/Samsung_T5/data/lobbying/oct8_postprocessed_filings.csv")
df[
    [
        "ccs_company",
        "ccs_description",
        "ccs_bills",
        "ccs_bills_number_only",
        "bills_with_ccs_terms",
        "definitely_ccs",
        "very_likely_ccs",
        "likely_ccs",
        "potentially_ccs",
    ]
].sum()

In [ ]:
maybe_df = df.loc[
    (df.contains_ccs_description == 1)
    & (df.ccs_bills == 0)
    & (df.ccs_bills_number_only == 0)
    & (df.ccs_company == 0)
    & (df.clean_h2_description == 0)
    & (df.clean_h2_company == 0)
]


maybe_df["obvi"] = [
    1
    if (
        (
            ("hydrogen" in x.lower())
            and (
                s
                in [
                    "clean hydrogen",
                    "oil and gas",
                    "pipelines",
                    "oilfield services",
                    "chemicals, refining, and plastics",
                ]
            )
        )
        | ("ccs" in x.lower())
        | ("capture co2" in x.lower())
        | ("carbon capute" in x.lower())
        | ("carbon capture" in x.lower())
        | ("carboncapture" in x.lower())
        | ("capture carbon dioxide" in x.lower())
        | ("carbon dioxide capture" in x.lower())
        | ("capture and sequestration of carbon dioxide" in x.lower())
        | ("capture and sequestration of co2" in x.lower())
        | ("capture and storage of carbon dioxide" in x.lower())
        | ("capture sequestration of carbon dioxide" in x.lower())
        | ("capture sequestration of co2" in x.lower())
        | ("capture storage of carbon dioxide" in x.lower())
        | ("renewable natural gas hydrogen" in x.lower())
        | ("co2 capture" in x.lower())
        | ("clean coal sequestration" in x.lower())
        | ("blue hydrogen" in x.lower())
        | ("clean hydrogen" in x.lower())
        | ("ccs" in x.lower())
        | ("ccus" in x.lower())
        | ("primacy" in x.lower())
        | ("capture and storage" in x.lower())
        | ("capture of carbon dioxide" in x.lower())
        | ("carbon sequestration" in x.lower())
        | ("carbon dioxide utilization" in x.lower())
        | ("carbon dioxide sequestration" in x.lower())
        | ("co2 sequestration" in x.lower())
        | ("co2 storage" in x.lower())
        | ("petra nova" in x.lower())
        | ("futuregen" in x.lower())
        | (("section 45" in x.lower()) and ("coal" in x.lower()))
        | ("enhanced oil recovery" in x.lower())
        | ("greenhouse gas capture and storage" in x.lower())
        | ("greenhouse gas capture storage" in x.lower())
        | (" eor " in x.lower())
        | ("carbon dioxide storage" in x.lower())
        | ("45q" in x.lower())
        | ("carbon storage" in x.lower())
        | ("45 q" in x.lower())
        | ("45v" in x.lower())
        | ("45 v" in x.lower())
        | ("45z" in x.lower())
        | ("45 z" in x.lower())
        | ("carbon utilization" in x.lower())
        | ("co2 geologic sequestration" in x.lower())
        | ("sequester co2" in x.lower())
        | ("injection of carbon dioxide" in x.lower())
        | ("captures and sequesters carbon dioxide" in x.lower())
        | ("sequester carbon dioxide" in x.lower())
        | ("storage of co2" in x.lower())
        | ("hydrogen hub" in x.lower())
        | ("carbon dioxide pipeline" in x.lower())
        | ("pipeline safety" in x.lower())
        | ("co2 pipeline" in x.lower())
        | ("carbon dioxide emmissions sequestration" in x.lower())
        | ("class vi" in x.lower())
        | ("renewable natural gas and hydrogen" in x.lower())
        | ("hydrogen tax credit" in x.lower())
        | ("carbon oxide capture" in x.lower())
        | ("carbon oxide sequestration" in x.lower())
        | ("conversion use and storage of carbon dioxide" in x.lower())
        | ("h2 hub" in x.lower())
    )
    else 0
    for x, s in zip(maybe_df.clean_description, maybe_df.sector)
]
print(maybe_df.obvi.value_counts())
for i, row in maybe_df.iterrows():
    if row["obvi"] == 0:
        print(f"{row['client_rename']}: {row['clean_description']}")
        for t in [
            "natural gas",
            "hydrogen",
            "air capture",
            "storage",
            "co2",
            "carbon dioxide",
            "injection",
            "pipeline",
            "co2 removal",
            "emissions",
            "pipe line",
            "oil recovery",
            "clean coal",
            "methane",
            "storage",
            "low carbon",
            "zero",
            "capture",
            "EPA",
            "UIC",
            "injection",
            "underground injection control",
            "inflation reduction act",
            "IIJA",
            "sequestration",
        ]:
            if t.lower() in row["clean_description"].lower():
                print(f"    {t}")

In [ ]:
maybe_df = df.loc[(df.definitely_ccs == 0) & (df.very_likely_ccs == 1)]

maybe_df["obvi"] = [
    1
    if (
        ("hydrogen" in x.lower())
        | ("sustainable aviation" in x.lower())
        | ("carbon management" in x.lower())
        | ("section 45" in x.lower())
    )
    else 0
    for x, s in zip(maybe_df.clean_description, maybe_df.sector)
]
print(maybe_df.obvi.value_counts())
print("")
for i, row in maybe_df.iterrows():
    if True:  # row["obvi"] == 0:
        print(f"{row['client_rename']}: {row['clean_description']}")
        for t in [
            "natural gas",
            "hydrogen",
            "air capture",
            "storage",
            "co2",
            "carbon dioxide",
            "injection",
            "pipeline",
            "co2 removal",
            "emissions",
            "pipe line",
            "oil recovery",
            "clean coal",
            "methane",
            "storage",
            "low carbon",
            "zero",
            "capture",
            "EPA",
            "UIC",
            "injection",
            "underground injection control",
            "inflation reduction act",
            "IIJA",
            "sequestration",
        ]:
            if t.lower() in row["clean_description"].lower():
                print(f"    {t}")

In [ ]:
# df.to_csv("~/Desktop/postprocessed_ccs.csv")

df = pd.read_csv("/Volumes/Samsung_T5/data/lobbying/oct8_postprocessed_filings.csv")
df[
    [
        "ccs_company",
        "ccs_description",
        "ccs_bills",
        "ccs_bills_number_only",
        "bills_with_ccs_terms",
        "definitely_ccs",
        "very_likely_ccs",
        "likely_ccs",
        "h2_mention_core_ff",
        "potentially_ccs",
    ]
].sum()

In [ ]:
df["n_indicators"] = df[
    [
        "ccs_bills",
        "ccs_company",
        "ccs_bills_number_only",
        "clean_hydrogen_company",
        "ccs_description",
        "bills_with_ccs_terms",
        "h2_mention_core_ff",
    ]
].sum(axis=1)

df[
    [
        "ccs_bills",
        "ccs_company",
        "ccs_bills_number_only",
        "clean_hydrogen_company",
        "ccs_description",
        "bills_with_ccs_terms",
        "h2_mention_core_ff",
    ]
].loc[df.n_indicators <= 1].sum()

In [ ]:
for i, row in df.loc[(df.n_indicators == 1) & (df.ccs_description == 1)].iterrows():
    if row["n_indicators"] <= 1:
        print(
            f"{i} ({row['client_rename']}, {row['client_name']}): {row['clean_description']}"
        )
        if row["ccs_bills"]:
            print("ccs_bills")
        elif row["ccs_company"]:
            print("ccs_company")
        elif row["clean_hydrogen_company"]:
            print("clean_hydrogen_company")
        elif row["ccs_description"]:
            print("ccs_description")
        elif row["ccs_bills_number_only"]:
            print("ccs_bills_number_only")

In [256]:
s = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/ccs_postproc_specifications.yml"
)["ccs_description"]
p = "Issues related to methane abatement capture policy HR3982 Methane Reduction and Economic Growth Act"
for ss in s:
    if terms_present(p, [ss]):
        print(ss)

In [12]:
df = df.loc[df.definitely_ccs == 1]

tot_dollars = df.lobbying_activity_usd.sum()

tot_contacts = df.n_entities_lobbied.sum()

df["pct_of_lobbying_dollars_definitely_ccs"] = [
    100 * x / tot_dollars for x in df.lobbying_activity_usd
]

df["pct_of_entity_contacts_definitely_ccs"] = [
    100 * x / tot_contacts for x in df.n_entities_lobbied
]

In [13]:
print(
    df.loc[
        (df.definitely_ccs == 1) & (df.filing_year > 2018)
    ].lobbying_activity_usd.sum()
)
print(df.loc[(df.definitely_ccs == 1)].lobbying_activity_usd.sum())

15367487.911531972
16656952.246885508


In [ ]:
print(
    df.loc[(df.definitely_ccs == 1) & (df.filing_year > 2018)].n_entities_lobbied.sum()
)
print(df.loc[(df.definitely_ccs == 1)].n_entities_lobbied.sum())

In [ ]:
33636 / (50 * 5 * 5.5)

In [ ]:
lobbying_by_sector_totals_df = (
    df[
        [
            "sector",
            "lumped_sector",
            "very_lumped_sector",
            "lobbying_activity_usd",
            "n_entities_lobbied",
            "pct_of_lobbying_dollars_definitely_ccs",
            "pct_of_entity_contacts_definitely_ccs",
        ]
    ]
    .loc[df["definitely_ccs"] == 1]
    .groupby(["very_lumped_sector"])
    .agg(
        {
            # "lumped_sector": "first",
            "very_lumped_sector": "first",
            "lobbying_activity_usd": sum,
            "n_entities_lobbied": sum,
            "pct_of_lobbying_dollars_definitely_ccs": sum,
            "pct_of_entity_contacts_definitely_ccs": sum,
        }
    )
    .sort_values(by="lobbying_activity_usd", ascending=False)
)


lobbying_by_sector_totals_df  # .to_clipboard()/

In [ ]:
core_ff_sectors = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/config_ccs_lda.yml"
)["core_ff_sectors"]
df.loc[
    df.sector.isin(core_ff_sectors) & (df.definitely_ccs == 1)
].pct_of_lobbying_dollars_definitely_ccs.sum()

In [ ]:
78 / 3

In [ ]:
lobbying_by_sector_totals_df = (
    df[
        [
            "sector",
            "lumped_sector",
            "very_lumped_sector",
            "lobbying_activity_usd",
            "n_entities_lobbied",
            "pct_of_lobbying_dollars_definitely_ccs",
            "pct_of_entity_contacts_definitely_ccs",
        ]
    ]
    .loc[df["definitely_ccs"] == 1]
    .groupby(["lumped_sector"])
    .agg(
        {
            # "lumped_sector": "first",
            "very_lumped_sector": "first",
            "lobbying_activity_usd": sum,
            "n_entities_lobbied": sum,
            "pct_of_lobbying_dollars_definitely_ccs": sum,
            "pct_of_entity_contacts_definitely_ccs": sum,
        }
    )
    .sort_values(by="lobbying_activity_usd", ascending=False)
)


lobbying_by_sector_totals_df  # .to_clipboard()

In [ ]:
df.columns.values

In [263]:
# merge client states
df.loc[df.client_state != df.client_ppb_state].client_state.value_counts(dropna=False)
df["client_state_merged"] = df.client_state.fillna(df["client_ppb_state"])

In [264]:
univ_df = pd.read_csv("~/Desktop/data/university_with_rename.csv", index_col=[0])
univ_df.institution_name = [x.upper() for x in univ_df.institution_name]
univ_df.to_csv("university_with_rename.csv")

In [ ]:
ccs_univ_df = (
    df[["client_rename", "lobbying_activity_usd"]]
    .loc[df.sector == "higher education"]
    .groupby(["client_rename"])
    .sum()
    .sort_values(by=["lobbying_activity_usd"], ascending=False)
)
univ_df.merge(
    ccs_univ_df, right_index=True, left_on="institution_name", how="outer"
).sort_values(by="total_og_funding_2010_to_2020", ascending=False)

In [ ]:
df[["filing_year", "client_rename", "lobbying_activity_usd"]].loc[
    df.definitely_ccs == 1
].groupby(["client_rename"]).sum().sort_values(
    # ].groupby(["filing_year", "client_rename"]).sum().sort_values(
    by=["lobbying_activity_usd"],
    ascending=False,
).head(
    20
)

In [ ]:
df[
    [
        "filing_year",
        "client_rename",
        "lobbying_activity_usd",
        "pct_of_lobbying_dollars_definitely_ccs",
        "n_entities_lobbied",
        "energy, dept of",
        "white house office",
        "executive office of the president (eop)",
        "president of the u.s.",
        "vice president of the u.s.",
        "office of the vice president of the united states",
        "environmental protection agency (epa)",
        "legistlative_entities_lobbied",
        "executive_entities_lobbied",
    ]
].loc[df.definitely_ccs == 1].groupby(["client_rename"]).sum().sort_values(
    # ].groupby(["filing_year", "client_rename"]).sum().sort_values(
    by=["lobbying_activity_usd"],
    ascending=False,
).head(
    18
).sum()

In [ ]:
# for histogram hue ordering:
sector_for_group = "very_lumped_sector"
very_lumped_order = list(
    df[[sector_for_group, "lobbying_activity_usd"]]
    .groupby(sector_for_group)
    .sum()
    .sort_values(by="lobbying_activity_usd", ascending=False)
    .index
)

# lobbying_activity_usd
years = list(range(2004, 2025))

tmp_df = (
    df[["filing_year", "lobbying_activity_usd", sector_for_group]]
    .loc[(df.definitely_ccs == 1)]
    .groupby([sector_for_group, "filing_year"])
    .sum()
    .sort_values(by="lobbying_activity_usd", ascending=False)
    .reset_index()
).copy(deep=True)
fig, ax = plt.subplots(1, 1, figsize=(17, 5))
ax = sns.histplot(
    data=tmp_df,
    x="filing_year",
    bins=[0.5 + x for x in years],
    hue=sector_for_group,
    hue_order=very_lumped_order,
    shrink=0.9,
    weights="lobbying_activity_usd",
    multiple="stack",
    palette="tab10",
)
ax.set_xticks(years)
ax.set_xticklabels([str(x) for x in years], fontsize=12)
ax.set_axisbelow(True)
ax.yaxis.grid()
ax.set_ylabel("100 million USD", fontsize=14)
ax.set_xlabel("")
# ax.legend(ncols=3, loc="upper center")a

In [ ]:
# lobbying_activity_usd
years = list(range(2004, 2025))
for agency in [
    "environmental protection agency (epa)",
    "energy, dept of",
    "treasury, dept of",
    "house of representatives",
    "senate",
]:
    tmp_df = (
        df[["filing_year", "lobbying_activity_usd", "very_lumped_sector"]]
        .loc[(df.very_likely_ccs == 1) & (df[agency] == 1)]
        .groupby(["very_lumped_sector", "filing_year"])
        .sum()
        .sort_values(by="lobbying_activity_usd", ascending=False)
        .reset_index()
    ).copy(deep=True)
    fig, ax = plt.subplots(1, 1, figsize=(17, 5))
    ax = sns.histplot(
        data=tmp_df,
        x="filing_year",
        bins=[0.5 + x for x in years],
        hue="very_lumped_sector",
        hue_order=very_lumped_order,
        shrink=0.9,
        weights="lobbying_activity_usd",
        multiple="stack",
    )
    ax.set_xticks(years)
    ax.set_xticklabels([str(x) for x in years], fontsize=12)
    ax.set_axisbelow(True)
    ax.yaxis.grid()
    ax.set_ylabel("100 million USD", fontsize=14)
    ax.set_xlabel("")
    ax.set_title(agency)
    # ax.legend(ncols=3, loc="upper center")a

In [ ]:
# lobbying_activity_usd
fig, ax = plt.subplots(1, 1, figsize=(17, 5))
years = list(range(2004, 2025))

tmp_df = (
    df[
        [
            "filing_year",
            "legistlative_entities_lobbied",
            "executive_entities_lobbied",
            "n_entities_lobbied",
        ]
    ]
    .loc[(df.very_likely_ccs == 1)]
    .groupby(["filing_year"])
    .sum()
    .reset_index()
).copy(deep=True)

tmp_df = tmp_df.melt(id_vars="filing_year")
ax = sns.lineplot(
    data=tmp_df,
    x="filing_year",
    # bins=[0.5 + x for x in years],
    # hue="very_lumped_sector",
    # shrink=0.9,
    y="value",
    hue="variable"
    # multiple="stack",
)
ax.set_xticks(years)
ax.set_xticklabels([str(x) for x in years], fontsize=12)
ax.set_axisbelow(True)
ax.yaxis.grid()
ax.set_ylabel("100 million USD", fontsize=14)
ax.set_xlabel("")
ax.legend()
# ax.legend(ncols=3, loc="upper center")a

In [ ]:
# lobbying_activity_usd
years = list(range(2004, 2025))

tmp_df = (
    df[["filing_year", "lobbying_activity_usd", "very_lumped_sector"]]
    .loc[(df.very_likely_ccs == 1)]  # & (cleaned_df.epa == 1)]
    .groupby(["very_lumped_sector", "filing_year"])
    .sum()
    .sort_values(by="lobbying_activity_usd", ascending=False)
    .reset_index()
).copy(deep=True)
fig, ax = plt.subplots(1, 1, figsize=(17, 5))
ax = sns.histplot(
    data=tmp_df,
    x="filing_year",
    bins=[0.5 + x for x in years],
    hue="very_lumped_sector",
    shrink=0.9,
    weights="lobbying_activity_usd",
    multiple="stack",
)
ax.set_xticks(years)
ax.set_xticklabels([str(x) for x in years], fontsize=12)
ax.set_axisbelow(True)
ax.yaxis.grid()
ax.set_ylabel("100 million USD", fontsize=14)
ax.set_xlabel("")
# ax.legend(ncols=3, loc="upper center")

In [ ]:
top_10 = [
    "OCCIDENTAL PETROLEUM",
    "SOUTHERN COMPANY",
    "EXXONMOBIL",
    "EDISON ELECTRIC INSTITUTE",
    "AMERICAN ELECTRIC POWER",
    "US CHAMBER OF COMMERCE",
    "SHELL",
    "DUKE",
    "BP AMERICA",
    "CHEVRON",
]
top_3 = ["EXXONMOBIL", "OCCIDENTAL PETROLEUM", "SOUTHERN COMPANY"]
top_5 = [
    "OCCIDENTAL PETROLEUM",
    "SOUTHERN COMPANY",
    "EXXONMOBIL",
    "EDISON ELECTRIC INSTITUTE",
    "AMERICAN ELECTRIC POWER",
]
top_df = (
    df[["lobbying_activity_usd", "n_entities_lobbied", "client_rename", "filing_year"]]
    .loc[(df.very_likely_ccs == 1) & (df.client_rename.isin(top_5))]
    .groupby(["filing_year", "client_rename"])
    .sum()
    .sort_values(by="lobbying_activity_usd", ascending=False)
)

years = list(range(2004, 2025))

fig, ax = plt.subplots(1, 1, figsize=(17, 5))
ax = sns.histplot(
    data=top_df,
    x="filing_year",
    bins=[0.5 + x for x in years],
    hue="client_rename",
    shrink=0.9,
    weights="lobbying_activity_usd",
    multiple="stack",
)
ax.set_xticks(years)
ax.set_xticklabels([str(x) for x in years], fontsize=12)
ax.set_axisbelow(True)
ax.yaxis.grid()
ax.set_ylabel("10 million USD", fontsize=14)
ax.set_xlabel("")
# ax.legend(ncols=3, loc="upper center")

In [ ]:
# top 50 pct companies chart
df[
    [
        "client_rename",
        "sector",
        "lobbying_activity_usd",
        "n_entities_lobbied",
        "pct_of_lobbying_dollars",
        "pct_of_entity_contacts",
    ]
].groupby(["client_rename"]).agg(
    {
        "lobbying_activity_usd": sum,
        "n_entities_lobbied": sum,
        "pct_of_lobbying_dollars": sum,
        "pct_of_entity_contacts": sum,
        "sector": "first",
    }
).sort_values(
    by="lobbying_activity_usd", ascending=False
).head(
    16
)  # .to_clipboard()

In [217]:
# top sectors chart
df[
    [
        "lumped_sector",
        "lobbying_activity_usd",
        "n_entities_lobbied",
        "pct_of_lobbying_dollars",
        "pct_of_entity_contacts",
    ]
].groupby(["lumped_sector"]).agg(
    {
        "lobbying_activity_usd": sum,
        "n_entities_lobbied": sum,
        "pct_of_lobbying_dollars": sum,
        "pct_of_entity_contacts": sum,
    }
).sort_values(
    by="lobbying_activity_usd", ascending=False
).to_clipboard()

In [ ]:
df[
    [
        "lumped_sector",
        "lobbying_activity_usd",
        "n_entities_lobbied",
        "pct_of_lobbying_dollars",
        "pct_of_entity_contacts",
    ]
].loc[(df.very_likely_ccs == 1) & (df.filing_year > 2018)].sum()

In [ ]:
27844 / (260 * 5.5)

In [87]:
df.groupby("client_rename").sum().to_clipboard()

In [ ]:
df.columns.values

In [95]:
def make_unique_list(x):
    return sorted(list(set(x)))

In [101]:
handling_dict = {
    "filing_uuid": make_unique_list,
    "n_lobbyists_for_activity": "sum",
    "filing_year": make_unique_list,
    "client_name": make_unique_list,
    #'client_general_description':'sum',
    "sector": "first",
    "lumped_sector": "first",
    "very_lumped_sector": "first",
    "lobbying_activity_usd": "sum",
    "n_entities_lobbied": "sum",
    "legistlative_entities_lobbied": "sum",
    "executive_entities_lobbied": "sum",
    "general_issue_code": make_unique_list,
    "pct_of_lobbying_dollars": "sum",
    "pct_of_entity_contacts": "sum",
} | dict(zip(entities, ["sum"] * len(entities)))

df.loc[df.likely_ccs == 1].groupby("client_rename").agg(handling_dict).to_clipboard()

In [ ]:
entities = [
    "u.s. trade representative (ustr)",
    "u.s. coast guard (uscg)",
    "securities & exchange commission (sec)",
    "defense department field activities",
    "federal motor carrier safety administration",
    "labor, dept of (dol)",
    "health & human services, dept of (hhs)",
    "overseas private investment corp (opic)",
    "pension benefit guaranty corporation (pbgc)",
    "u.s. customs & border protection",
    "council on environmental quality (ceq)",
    "farm credit administration",
    "advisory council on historic preservation (achp)",
    "federal deposit insurance corporation (fdic)",
    "bureau of industry and security (bis)",
    "bureau of reclamation",
    "centers for disease control & prevention (cdc)",
    "commerce, dept of (doc)",
    "office of policy development",
    "u.s. chemical safety & hazard investigation board",
    "u.s. postal service (usps)",
    "defense information systems agency",
    "natl labor relations board (nlrb)",
    "natl park service (nps)",
    "u.s. agency for international development (usaid)",
    "u.s. copyright office",
    "office of juvenile justice & delinquency prevention",
    "office of the secretary of defense",
    "natl science foundation (nsf)",
    "minority business development agency",
    "tennessee valley authority (tva)",
    "federal reserve system",
    "federal election commission (fec)",
    "natl credit union administration (ncua)",
    "defense intelligence agency (dia)",
    "appalachian regional commission",
    "u.s. fish & wildlife service (usfws)",
    "executive office of the president (eop)",
    "defense security cooperation agency (dsca)",
    "food & drug administration (fda)",
    "natl endowment for the humanities",
    "general services administration (gsa)",
    "energy, dept of",
    "federal railroad administration",
    "natl institute of justice",
    "federal energy regulatory commission (ferc)",
    "defense commissary agency",
    "veterans affairs, dept of (va)",
    "council of economic advisers (cea)",
    "federal emergency management agency (fema)",
    "agency for healthcare research & quality (ahrq)",
    "air force, dept of",
    "railroad retirement board",
    "amtrak - natl railroad passenger corporation",
    "office of the vice president of the united states",
    "african development foundation",
    "natl guard",
    "minerals management service",
    "federal bureau of investigation (fbi)",
    "internal revenue service (irs)",
    "agriculture, dept of (usda)",
    "consumer product safety commission (cpsc)",
    "natl mediation board",
    "bureau of the census",
    "undetermined",
    "state, dept of (dos)",
    "natl security council (nsc)",
    "congressional budget office (cbo)",
    "employment & training administration",
    "federal aviation administration (faa)",
    "homeland security, dept of (dhs)",
    "health resources & services administration (hrsa)",
    "u.s. trade & development agency (ustda)",
    "library of congress (loc)",
    "defense logistics agency",
    "surface transportation board (stb)",
    "pension & welfare benefits administration",
    "economic development administration",
    "white house office",
    "natl institute of standards & technology (nist)",
    "bureau of citizenship & immigration services (bcis)",
    "vice president of the u.s.",
    "office of administration",
    "natl economic council (nec)",
    "natl aeronautics & space administration (nasa)",
    "small business administration (sba)",
    "financial crimes enforcement network (fincen)",
    "export-import bank of the united states (exim bank)",
    "navy, dept of",
    "office of surface mining reclamation & enforcement",
    "agency for toxic substances & disease registry",
    "patent & trademark office (pto)",
    "federal housing finance agency (fhfa)",
    "bureau of land management (blm)",
    "centers for medicare and medicaid services (cms)",
    "natl telecommunications & information administration (ntia)",
    "natl transportation safety board (ntsb)",
    "election assistance commission (eac)",
    "education, dept of",
    "treasury, dept of",
    "occupational safety & health administration (osha)",
    "u.s. immigration & customs enforcement (ice)",
    "government accountability office (gao)",
    "social security administration (ssa)",
    "natl institute of mental health (nimh)",
    "commodity futures trading commission (cftc)",
    "merit systems protection board",
    "marine mammal commission",
    "administration for children & families (acf)",
    "defense, dept of (dod)",
    "defense advanced research projects agency (darpa)",
    "substance abuse & mental health services administration (samhsa)",
    "natl indian gaming commission",
    "office of special counsel",
    "u.s. geological survey (usgs)",
    "bureau of labor statistics (bls)",
    "alcohol & tobacco tax & trade bureau (ttb)",
    "interior, dept of (doi)",
    "natl institutes of health (nih)",
    "drug enforcement administration (dea)",
    "federal transit administration (fta)",
    "mine safety & health administration (msha)",
    "maritime administration (marad)",
    "transportation security administration (tsa)",
    "office of personnel management (opm)",
    "u.s. forest service",
    "indian health service",
    "federal accounting standards advisory board (fasab)",
    "pipeline & hazardous materials safety administration",
    "bureau of indian affairs (bia)",
    "house of representatives",
    "natl oceanic & atmospheric administration (noaa)",
    "defense contract audit agency (dcaa)",
    "office of management & budget (omb)",
    "army, dept of (other)",
    "intl trade administration (ita)",
    "nuclear regulatory commission (nrc)",
    "defense finance & accounting service (dfas)",
    "st. lawrence seaway development corporation",
    "director of national intelligence (odni)",
    "housing & urban development, dept of (hud)",
    "u.s. architectural & transportation barriers compliance board (access)",
    "u.s. international trade commission (itc)",
    "veterans employment & training service",
    "u.s. marines",
    "federal communications commission (fcc)",
    "natl security agency (nsa)",
    "natl highway traffic safety administration (nhtsa)",
    "federal trade commission (ftc)",
    "equal employment opportunity commission (eeoc)",
    "office of science & technology policy (ostp)",
    "postal regulatory commission",
    "senate",
    "transportation, dept of (dot)",
    "federal maritime commission",
    "environmental protection agency (epa)",
    "office of the comptroller of the currency (occ)",
    "consumer financial protection bureau (cfpb)",
    "army, dept of (corps of engineers)",
    "federal highway administration (fha)",
    "justice, dept of (doj)",
    "president of the u.s.",
    "federal mediation & conciliation service",
    "risk management agency",
    "u.s. information agency (usia)",
]